In [3]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz --user
nlp = en_core_web_sm.load()
import pandas as pd
import json
!pip install bs4 --user
from bs4 import BeautifulSoup
import requests
import re
import os
import mmap
import random
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from spacy.util import minibatch, compounding

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Load data

In [4]:
with open('data.json', 'r') as f:
    dict_json_weighted = json.load(f)
object_1 = list(dict_json_weighted.keys())[2]
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(str(i)+"-"+dict_json_weighted[object_1]['log'][i]['text'])

0- I would like to find somewhere with expensive Kosher food.
1-I'm not seeing any expensive Kosher restaurants, would you like to try a different price range perhaps?
2-Sure. I also need the restaurant to be in the center.
3-Sorry it looks like there are no kosher restaurants in any price range in town at all. Would you like another food type?
4-Sure, let's try Italian instead. 
5-There are 4 expensive Italian restaurants in the center of town. I suggest Caffe Uno at 32 Bridge Street City Centre. Would you like to make a reservation? 
6-Caffe Uno sounds great. Can I get their phone number, please? 
7-Their phone number is 01223448620. Is there anything else I can help you with?
8-No, you've been very helpful. Thank you!  Good bye.
9-Thank you, goodbye.


## Load ontology

In [5]:
with open('ontology.json', 'r') as o:
    ontology = json.load(o)
    my_ent=list(ontology.keys())
    
    
action=['inform', 'request', 'select', 'recommend', 'not found', 'request booking', 'offer booking', 
        'inform booked', 'decline booking', 'welcome','greet', 'bye', 'reqmore' ]
topic= ['hotel', 'restaurant', 'taxi', 'train', 'attraction', 'police', 'hospital']

## display topics and actions

In [6]:
top=[]
act=[]
ont=list(ontology.keys())
for i in range(len(dict_json_weighted[object_1]['log'])):
    #print(dict_json_weighted[object_1]['log'][i]['text'])
    sentence =nlp (dict_json_weighted[object_1]['log'][i]['text'])
    for x in [y for y in sentence if y.pos_ != 'PUNCT']:
        if str(x).lower() in topic:
            if str(x).lower() not in top:
                top.append(str(x).lower()) 
        if x.lemma_ in action:
            if x.lemma_ not in act:
                act.append(x.lemma_)
        #for z in ont:
            #if str(x) in str(ontology[z]):
                #print(z+" :"+str(x))           
print("topics: "+str(top))
print("actions: "+str(act))


topics: ['restaurant']
actions: ['bye']


## Display the updated slots 

In [7]:
for i in dict_json_weighted[object_1]['goal']:
    if i not in['message', 'topic']:
        if dict_json_weighted[object_1]['goal'][i] !={}:
            print("****** "+i+" ******")
            for j in (dict_json_weighted[object_1]['goal'][i]):
                if dict_json_weighted[object_1]['goal'][i][j] not in ['fail_info', {}]:
                    print("°"+j)
                    print(dict_json_weighted[object_1]['goal'][i][j])
    

****** restaurant ******
°fail_info
{'pricerange': 'expensive', 'food': 'kosher', 'area': 'centre'}
°reqt
['phone']
°info
{'pricerange': 'expensive', 'food': 'italian', 'area': 'centre'}


# Custom NER

### create training sample

In [15]:
Train_data = [
     (
        "Hello, I am looking for a cheap place to dine in the centre", { "entities" :
        [(26, 31, "restaurant-price-range"), (45, 58, "restaurant-area")]
        }
    ),(
        "not really but i need it for 3 people",  { "entities" : 
        [(29, 37, "restaurant-book-people")]
        }
    ),(
        "I recommend La Raza - fantastic (and cheap!) Spanish cuisine. I'd be happy to book a table for you, if you like.",  { "entities" : 
        [(12, 20, "restaurant-name"), (38, 43, "restaurant-price-range"),(46, 61, "restaurant-food")]
        }

    ),(
        "Yes for 3 at 14:45 wednesday, can I get the reference number too please", 
        { "entities" :
        [(4, 9, "restaurant-book-people"), (10, 18, "restaurant-book-time"), (19, 27, "restaurant-book-day")]
        }
    ),( 
        "Both offer free parking and free wifi. The Alexander B&B is 4 star rated. The El Shaddai has a 0 rating.", { "entities" : 
             [(11, 23, "hotel-parking"), (28, 37, "hotel-internet"), (39, 56, "hotel-name"), (60, 66, "hotel-stars"), (74, 87, "hotel-name"), (95, 103, "hotel-stars")]
             }
    ),( 
        "3 nights, 3 people, please.", { "entities" : 
             [(0, 8, "hotel-book-stay"), (10, 18, "hotel-book-people")]
             }
    ),( 
        "I would like a taxi from the hotel to the restaurant. I would like to arrive by 14:45. I would like the contact number and car type",  { "entities" : 
             [(77, 82, "taxi-arrive-by")]
             }
    ),( 
        "Thanks. I also need a cheap place to stay in the center.", { "entities" : 
             [(22, 33, "hotel-price-range"),(42, 55, "hotel-area")]
             }
    ),( 
        "I am looking for info about a hotel called city centre north b and b. Can you help?", { "entities" : 
             [(43, 67, "hotel-name")]
             }
    ),( 
        "Yes. City Centre North B and B is a guesthouse in the north side of town, that has both free internet and parking. Do you want to book a room?", { "entities" : 
             [(5,30, "hotel-name"),(36, 46, "hotel-type"),(47, 71, "hoyel-area")]
             }
    ),( 
        "Ok, I am also looking for a train from Cambridge to Bishops Stortford.", { "entities" : 
             [(34, 48, "train-departure"),(49, 69, "train-destination")]
             }
    ),( 
        "I'll be leaving Monday after 15:30.", { "entities" : 
             [(16, 22, "train-day"),(23, 34, "train-leave-at")]
             }
    ),( 
        "TR2630 leaves at 17:29. Would you like for me to book the train for you?", { "entities" : 
             [(14, 22, "train-leave-at")]
             }
    ),( 
        "Yes I would like to book the train for 4 people and I'll need the reference number please.", { "entities" : 
             [(38, 46, "train-book-people")]
             }
    ),( 
        "I'm looking for a Mediterranean restaurant near the centre of town.", { "entities" : 
             [(18, 31, "restaurant-food"), (42, 65, "restaurant-area")]
             }
    ),( 
        "I've got three that fit that bill... the gardenia, la mimosa, and shiraz.  Do you have a preference?", { "entities" : 
             [(36, 48, "restaurant-name"),(50, 59, "restaurant-name"),(65, 71, "restaurant-name")]
             }
    ),( 
        "Great, could I book a table of 5 for 11:30 on wednesday? If so, please provide me with a reference number", { "entities" : 
             [(20, 32, "resraurant-book-people"),(33, 42, "restaurant-book-time"),(43, 55, "restaurant-bookday")]
             }
    ),( 
        "Is one of them in the cheap price range? ", { "entities" : 
             [(22, 27, "restaurant-price-range")]
             }
    ),( 
        "Yes. Can you please search for an attraction in the type of college?", { "entities" : 
             [(45, 67, "attraction-type")]
             }
    ),( 
        "Is an entrance fee okay? If so, the corpus christi college located at king's parade. Fee is 2 pounds. Would you like the phone number? ", { "entities" : 
             [(32, 57, "attraction-name")]
             }
    ),( 
        "I would like to leave Corpus Christi College in enough time to arrive at the restaurant at the booked time of 11:30. ", { "entities" : 
             [(22, 44, "train-departure"), (81, 101, "train-arrive-by")]
             }
    ),( 
        "I'm looking for a place to stay in Cambridge. It should include free wifi and be a 3 star.  Can you help me please? ", { "entities" : 
             [(62, 71, "hotel-internet"), (79, 87, "hotel-stars")]
             }
    ),( 
        "There are five to choose from. Would you like moderate or expensive price point? ", { "entities" : 
             [(45, 52, "hotel-price-range"), (56, 64, "hotel-price-range")]
             }
    ),( 
        "I have the gonville hotel. It is expensive. Would you like me to book that for you?", { "entities" : 
             [(7, 25, "hotel-name"), (31, 42, "hotel-price-range")]
             }
    ),( 
        "Do they have free parking? If so, please reserve a room for Tuesday for 7 people, 4 nights.", { "entities" : 
             [(13, 25, "hotel-parking"), (55, 66, "hotel-book-day"), (67, 79, "hotel-book-people"), (81,89, "hotel-book-stay")]
             }
    ),
]



### Training

In [27]:
output_dir2 = ("output_dir2")
def main_train(model=None, output_dir=r'model', n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in Train_data:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    #for i in my_ent:
     #   ner.add_label(i)

    # get names of other pipes to disable them during training
    #other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    #with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
    if model is None:
        nlp.begin_training()
    else:
        nlp.resume_training()
    #if model is None:
     #   optimizer = nlp.begin_training()
    #else:
     #   optimizer = nlp.entity.create_optimizer()
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER

        for itn in range(n_iter):
            print("Starting iteration" + str(itn))
            random.shuffle(Train_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(Train_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.3,  # dropout - make it harder to memorise data
                losses=losses,
                )
                
            print("Losses", losses)
    new_model=nlp
    # test the trained model
    for text, _ in Train_data:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    #if output_dir2 is not None:
     #   output_dir2 = Path(output_dir2)
      #  if not output_dir2.exists():
       #     output_dir2.mkdir()
    nlp.to_disk(output_dir2)
    print("Saved model to", output_dir2)
main_train(model=None, output_dir=r'model', n_iter=100)

Created blank 'en' model
Starting iteration0
Losses {'ner': 407.04091942310333}
Starting iteration1
Losses {'ner': 173.47640916365137}
Starting iteration2
Losses {'ner': 75.66361324422658}
Starting iteration3
Losses {'ner': 57.45560031805144}
Starting iteration4
Losses {'ner': 60.02353236267663}
Starting iteration5
Losses {'ner': 55.59217436632565}
Starting iteration6
Losses {'ner': 43.56975348384276}
Starting iteration7
Losses {'ner': 41.36164258580454}
Starting iteration8
Losses {'ner': 52.5289842049358}
Starting iteration9
Losses {'ner': 52.524293600982325}
Starting iteration10
Losses {'ner': 52.410524048581706}
Starting iteration11
Losses {'ner': 72.7649491301272}
Starting iteration12
Losses {'ner': 61.13251814182745}
Starting iteration13
Losses {'ner': 54.78065699482503}
Starting iteration14
Losses {'ner': 43.624842041492485}
Starting iteration15
Losses {'ner': 43.889732429956695}
Starting iteration16
Losses {'ner': 72.02679354255822}
Starting iteration17
Losses {'ner': 71.6786850

### Testing the model

In [17]:
def test_model(nlp):
    texts = [
        "Yes I would like to book the train for 4 people and I'll need the reference number please.",
        "I've got three that fit that bill... the gardenia, la mimosa, and shiraz.  Do you have a preference?",
        "Great, could I book a table of 5 for 11:30 on wednesday?",
    ]
    docs = nlp.pipe(texts)
    for doc in docs:
        print(doc.text)
        print([(t.text, t.dep_, t.head.text) for t in doc if t.dep_ != "-"])

In [18]:
test_model(nlp)

Yes I would like to book the train for 4 people and I'll need the reference number please.
[('Yes', 'intj', 'like'), ('I', 'nsubj', 'like'), ('would', 'aux', 'like'), ('like', 'ROOT', 'like'), ('to', 'aux', 'book'), ('book', 'xcomp', 'like'), ('the', 'det', 'train'), ('train', 'dobj', 'book'), ('for', 'prep', 'book'), ('4', 'nummod', 'people'), ('people', 'pobj', 'for'), ('and', 'cc', 'like'), ('I', 'nsubj', 'need'), ("'ll", 'aux', 'need'), ('need', 'conj', 'like'), ('the', 'det', 'number'), ('reference', 'compound', 'number'), ('number', 'dobj', 'need'), ('please', 'intj', 'need'), ('.', 'punct', 'need')]
I've got three that fit that bill... the gardenia, la mimosa, and shiraz.  Do you have a preference?
[('I', 'nsubj', 'got'), ("'ve", 'aux', 'got'), ('got', 'ROOT', 'got'), ('three', 'dobj', 'got'), ('that', 'nsubj', 'fit'), ('fit', 'relcl', 'three'), ('that', 'det', 'bill'), ('bill', 'dobj', 'fit'), ('...', 'punct', 'got'), ('the', 'det', 'gardenia'), ('gardenia', 'ROOT', 'gardenia')

In [19]:
print("Loading from", output_dir2)
nlp2 = spacy.load(output_dir2)
doc = nlp2("3 nights, 3 people, please.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
displacy.render(doc,  style='ent')

Loading from output_dir2
Entities [('3 nights', 'hotel-book-stay'), ('3 people', 'hotel-book-people')]


In [20]:
print("Loading from", output_dir2)
nlp2 = spacy.load(output_dir2)
doc = nlp2("Do they have free parking? If so, please reserve a room for Tuesday for 7 people, 4 nights.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
displacy.render(doc,  style='ent')

Loading from output_dir2
Entities [('free parking', 'hotel-parking')]


In [25]:
print("Loading from", output_dir2)
nlp2 = spacy.load(output_dir2)
doc = nlp2("I want to reserve a room for 4 nights.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
displacy.render(doc,  style='ent')

Loading from output_dir2
Entities []


/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [22]:
print("Loading from", output_dir2)
nlp2 = spacy.load(output_dir2)
doc = nlp2("Hello, I am looking for some places to go in the centre of town. There is a lot to do in the centre. Is there a certain type of attraction you'd like to see or do? I was thinking something related to multiple sports We don't have any multiple sports facilities in the centre of town. Do you want to look for something else? How about a theatre? I think we would enjoy seeing a show! May I recommend Cambridge Arts Theatre located in the centre of town? That would be great. Can I have the postcode, address, and entrance fee please? I don't have any information about admission, but you can call them to get more information.  The number is 01223503333. The address is 6 saint edward's passage cb23pj Ok.  I also need info on the restaurant Darrys cookhouse and wine shop. Darrys Cookhouse and Wine Shop is a modern European restaurant located in the centre of town.  It's in the expensive price range.  Would you like to make a reservation? Yes, that would be great. I'll need a table for 7 on Friday at 18:00.  Booking was successful. The table will be reserved for 15 minutes. Reference number is : BT75ESXM. Can I help you with anything else? I don't think so.  Thanks for all your help. Hope you have a great time during your trip. That was so nice of you to say thanks so much. You're very welcome! Take care!")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
displacy.render(doc,  style='ent')

Loading from output_dir2
Entities [('in the centre of', 'attraction-type'), ('in the centre', 'hotel-area'), ('certain type', 'hotel-price-range'), ('in the centre', 'hotel-area'), ('May I', 'hotel-stars'), ('Cambridge', 'restaurant-food'), ('Arts Theatre located', 'train-destination'), ('in the centre of', 'attraction-type'), ('the postcode', 'restaurant-bookday'), ('The number is 01223503333', 'hotel-area'), ('The address', 'hotel-stars'), ('6 saint', 'hotel-stars'), ('on the restaurant', 'hotel-area'), ('Darrys cookhouse and', 'hotel-internet'), ('Darrys Cookhouse and', 'train-departure'), ('modern', 'hotel-type'), ('European restaurant located', 'hotel-area'), ('in the centre', 'hotel-area'), ('in the expensive', 'hotel-area'), ('?', 'hotel-type'), ('on Friday', 'restaurant-book-people'), ('at 18:00', 'restaurant-book-time'), ('Booking was successful', 'train-destination'), ('The table', 'hotel-stars'), ('for 15 minutes.', 'resraurant-book-people'), (': BT75ESXM', 'hotel-stars')]


In [29]:
print("Loading from", output_dir2)
nlp2 = spacy.load(output_dir2)
doc = nlp2("I am looking for the addresses of each of the moderately priced restaurants in the south part of town. The address for Pizza Hut Cherry Hinton is G4 Cambridge Leisure Park Clifton Way Cherry Hinton and the address for Restaurant Alimentum is 152 - 154 Hills Road. Is there anything else? There is restaurant alimentum which is a modern european restaurant located in the south part of town and is in the moderate price range. Could you give me the address for Restaurant Alimentum? Restaurant Alimentum's address is G4 Cambridge Leisure Park Clifton Way Cherry Hinton. May I help you with anything else? No thank you good bye Thank you for using our system. Goodbye.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
displacy.render(doc,  style='ent')

Loading from output_dir2
Entities [('Hinton', 'restaurant-food'), ('Cambridge Leisure', 'hotel-internet'), ('Park Clifton Way Cherry Hinton and the', 'hotel-name'), ('Restaurant Alimentum is', 'hotel-name'), ('154 Hills', 'hotel-stars'), ('alimentum which', 'hotel-stars'), ('european', 'train-day'), ('in the south', 'hotel-area'), ('in the moderate', 'hotel-area'), ('for Restaurant Alimentum', 'hotel-area'), ('Cambridge Leisure Park Clifton Way Cherry Hinton', 'hotel-name')]


### Model evaluation

In [28]:
def evaluate(model, examples):
    scorer = Scorer()
    for text, metadata in Train_data:
        #print(text)
        doc_gold_text = model.make_doc(text)
        gold = GoldParse(doc_gold_text, entities=metadata['entities'])
        pred_value = model(text)
        scorer.score(pred_value, gold)
    return scorer.scores
test_data=[]
x=[]
object_2 = list(dict_json_weighted.keys())[6]
#for i in dict_json_weighted[object_2]['goal']:
#    if i!='message':
#        if dict_json_weighted[object_2]['goal'][i] !={}:
#            print("****** "+i+" ******")
#            for j in (dict_json_weighted[object_2]['goal'][i]):
#                if dict_json_weighted[object_2]['goal'][i][j] not in ['fail_info', {}, False]:
#                    print("°"+j)
#                    print(dict_json_weighted[object_2]['goal'][i][j])
                    #x.append({j ,(dict_json_weighted[object_2]['goal'][i][j])})
            
for i in range(len(dict_json_weighted[object_2]['log'])):
    #print(dict_json_weighted[object_2]['log'][i])
    #test_text=test_text+"\n"+dict_json_weighted[object_2]['log'][i]['text']
    #test_annot=test_annot+"\n"+str(dict_json_weighted[object_2]['log'][i]['metadata'])
    if (dict_json_weighted[object_2]['log'][i]['metadata'])!={} :
        test_data.append({dict_json_weighted[object_2]['log'][i]['text'], str(dict_json_weighted[object_2]['log'][i]['metadata'])})
        print(dict_json_weighted[object_2]['log'][i]['metadata'])
#print(test_data)
test_result = evaluate(new_model, test_data)

{'train': {'semi': {'day': '', 'leaveAt': '', 'destination': '', 'departure': '', 'arriveBy': ''}, 'book': {'people': '', 'booked': []}}, 'restaurant': {'semi': {'name': '', 'pricerange': '', 'food': '', 'area': ''}, 'book': {'day': '', 'people': '', 'booked': [], 'time': ''}}, 'attraction': {'semi': {'type': 'not mentioned', 'name': 'not mentioned', 'area': 'not mentioned'}, 'book': {'booked': []}}, 'hospital': {'semi': {'department': ''}, 'book': {'booked': []}}, 'taxi': {'semi': {'leaveAt': '', 'arriveBy': '', 'destination': '', 'departure': ''}, 'book': {'booked': []}}, 'hotel': {'semi': {'area': '', 'stars': '', 'type': '', 'pricerange': '', 'parking': '', 'name': '', 'internet': ''}, 'book': {'day': '', 'people': '', 'booked': [], 'stay': ''}}, 'bus': {'semi': {'day': '', 'leaveAt': '', 'destination': '', 'departure': '', 'arriveBy': ''}, 'book': {'people': '', 'booked': []}}, 'police': {'semi': {}, 'book': {'booked': []}}}
{'train': {'semi': {'day': '', 'leaveAt': '', 'destinati

NameError: name 'new_model' is not defined